#Predicting Median Income in the United States by County using a Bayesian Approach

In [ ]:
# Imports
%matplotlib inline
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
from scipy import stats
from scipy.special import erf
import pandas as pd
import seaborn as sns
az.style.use('arviz-darkgrid')
plt.rcParams['figure.dpi'] = 300
import warnings
warnings.filterwarnings('ignore')
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')
az.style.use('arviz-darkgrid')
random_seed = 5192024

In [ ]:
df = pd.read_csv('QOL(County Level).csv')
df

In [ ]:
df = df.drop(['Overall Rank', 'ParkScore2023 Rank', '%CvgCityPark'], axis=1)

In [ ]:
df.dtypes
cols_comma = ['2022 Population']
cols_na_mean = ['WaterQualityVPV', 'Stu:Tea Rank', 'Unemployment']
perc = ['Unemployment','2020 PopulrMajor%','%CvgStatePark', 'AQI%Good', 'AVG C2I','1p0c','1p1c','1p2c','1p3c','1p4c','2p0c','2p1c','2p2c','2p3c','2p4c']
com_dollar = ['Cost of Living', '2022 Median Income']

In [ ]:
# Function to remove commas and convert to int
def remove_commas_and_convert_to_int(value):
    try:
        return int(value.replace(',', ''))
    except ValueError:
        # Handle cases where conversion is not possible
        return value

# Apply the function to the specified column
df['2022 Population'] = df['2022 Population'].apply(remove_commas_and_convert_to_int)


# percentages
for x in perc:
  df[x] = df[x].str.replace('%', '').astype(float)

# dollar sign and comma
def clean_and_convert_to_int(value):
    try:
        # Remove commas and dollar signs, then convert to int
        return int(value.replace(',', '').replace('$', ''))
    except ValueError:
        # Handle cases where conversion is not possible
        return value

# Apply the function to the specified column
for x in com_dollar:
  df[x] = df[x].replace('[\$,]', '', regex=True).astype(float)


# 2016 crime rate (fraction?)

def convert_fraction_to_float(fraction_str):
    if '/' in fraction_str:
        numerator, denominator = map(int, fraction_str.split('/'))
        return numerator / denominator
    else:
        # Handle the case when the input is a single number (e.g., '0')
        return float(fraction_str)

# Apply the function to the specified column
df['2016 Crime Rate'] = df['2016 Crime Rate'].apply(convert_fraction_to_float)
# Print the updated DataFrame
# Calculate the mean of non-NaN values in the column
for x in cols_na_mean:
  df[x] = df[x].replace(-1, df[x].mean())

df['%CvgStatePark'] = df['%CvgStatePark'].replace(-100, 0)
df

In [ ]:
df.hist(figsize=(15, 10))

## Addressing Categorical Data

In [ ]:
categorical_columns = ['countyhelper', 'LSTATE', 'NMCNTY', 'FIPS', 'LZIP',
                      'ULOCALE', '2020PopulrVoteParty']
for x in categorical_columns:
    df[x] = df[x].astype('category')

In [ ]:
df.dtypes

In [ ]:
df_select = df[['countyhelper', 'LSTATE', 'ULOCALE',
       '2022 Population', '2016 Crime Rate', 'Unemployment',
       '2020PopulrVoteParty', '2020 PopulrMajor%', 'AQI%Good',
       'WaterQualityVPV', 'NtnlPrkCnt', '%CvgStatePark', 'Cost of Living',
       '2022 Median Income', 'Stu:Tea Rank',
       'Diversity Rank (Race)', 'Diversity Rank (Gender)']]

##Numerical EDA

In [ ]:
df_eda_num = df_select[['2022 Population', '2016 Crime Rate', 'Unemployment',
       '2020 PopulrMajor%', 'AQI%Good',
       'WaterQualityVPV', 'NtnlPrkCnt', '%CvgStatePark', 'Cost of Living',
       'Stu:Tea Rank', 'Diversity Rank (Race)', 'Diversity Rank (Gender)',
        '2022 Median Income']]

In [ ]:
df_eda_num_reduced = df_eda_num.iloc[0:1000,:]

In [ ]:
pairplot = sns.pairplot(df_eda_num_reduced)
pairplot

In [ ]:
pairplot.savefig('Income_pairplot')

##Categorical EDA

In [ ]:
ax = df_select['2020PopulrVoteParty'].value_counts().plot(kind='bar',
                                    figsize=(5,4),
                                    title="County Count By Popular Party")
ax.set_xlabel("Political Party")
ax.set_ylabel("Count")

In [ ]:
df_pol = df.groupby('2020PopulrVoteParty').agg({'2022 Median Income': 'mean'}).reset_index()

In [ ]:
df_pol.plot(x="2020PopulrVoteParty", y="2022 Median Income", kind="bar")

In [ ]:
df_plot = df_select.groupby(['ULOCALE', 'LSTATE']).size().reset_index().pivot(columns='ULOCALE', index='LSTATE', values=0)

In [ ]:
df_plot.plot(kind='bar', stacked=True, figsize=(18,7), xlabel = 'State', ylabel='County Count')

In [ ]:
df_descr = df.groupby(['ULOCALE']).agg({'2022 Median Income': 'mean'}).reset_index()

In [ ]:
df_descr.plot(x="ULOCALE", y="2022 Median Income", kind="bar")

In [ ]:
df_state = df.groupby('LSTATE').agg({'2022 Median Income': 'mean'}).reset_index()

In [ ]:
df_state.plot(x="LSTATE", y="2022 Median Income", kind="bar", figsize=(12,6))

##Further Data Processing: Categorical Variables

In [ ]:
df_select = pd.get_dummies(df_select, drop_first = False,
                              columns = ['LSTATE', 'ULOCALE'], dtype = float)

In [ ]:
df_select['2020PopulrVoteParty'] = pd.Categorical(df_select['2020PopulrVoteParty']).codes

In [ ]:
df_select = df_select.set_index('countyhelper')

In [ ]:
df_select.head()

In [ ]:
df_select.columns

##Splitting Data into Test and Train Sets

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_select, test_size=0.2, random_state=2023)
train

In [ ]:
test

##Log Transformations

In [ ]:
median_income_train_log = np.log(train['2022 Median Income'])
median_income_test_log = np.log(test['2022 Median Income'])
train['Cost of Living Log'] = np.log(train['Cost of Living'])
test['Cost of Living Log'] = np.log(test['Cost of Living'])
test

In [ ]:
train.drop(['2022 Median Income', 'Cost of Living'], axis=1, inplace = True)
test.drop(['2022 Median Income', 'Cost of Living'], axis=1, inplace = True)

##Further Data Processing: Scaling Numerical Variables

##Train

In [ ]:
numeric_var = ['2022 Population', '2016 Crime Rate', 'Unemployment', '2020 PopulrMajor%', 'AQI%Good',
       'WaterQualityVPV', 'NtnlPrkCnt', '%CvgStatePark', 'Cost of Living Log',
       'Stu:Tea Rank', 'Diversity Rank (Race)','Diversity Rank (Gender)']
scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(train[numeric_var])
stats.describe(X_scaled_train)

In [ ]:
categorical_var = list(set(train.columns) - set(numeric_var))
X_income_train = np.hstack((X_scaled_train, train[categorical_var].to_numpy()))

In [ ]:
X_income_train.shape

## Test

In [ ]:
numeric_var = ['2022 Population', '2016 Crime Rate', 'Unemployment',
       '2020 PopulrMajor%', 'AQI%Good', 'WaterQualityVPV', 'NtnlPrkCnt',
       '%CvgStatePark', 'Cost of Living Log', 'Stu:Tea Rank', 'Diversity Rank (Race)',
       'Diversity Rank (Gender)']
scaler = StandardScaler()
X_scaled_test = scaler.fit_transform(test[numeric_var])
stats.describe(X_scaled_test)

In [ ]:
categorical_var = test.iloc[:,11:75].columns
X_income_test = np.hstack((X_scaled_test, test[categorical_var].to_numpy()))

In [ ]:
X_income_test.shape

## Model Building:

##Full Model

In [ ]:
# build the linear model

median_income_train_log_mean = median_income_train_log.mean()
k = X_income_train.shape[1]

with pm.Model() as income_lin_model:

    #Data
    X = pm.MutableData("X", X_income_train) # training data
    y = pm.MutableData("y", median_income_train_log)

    #Regression model parameters
    α = pm.Normal('α', mu = median_income_train_log_mean, sigma = 17000) # uninformed prior
    β = pm.MvNormal('β', mu=np.zeros(k), cov= np.eye(k), shape=k, dims ='Predictors') # informed priors

    #Likelihood parameters
    nu = pm.Exponential('nu', 1/29.0)+ 1 # uninformed prior
    σ = pm.HalfStudentT('σ',2,8)
    μ = α + pm.math.dot(X,β)
    log_income = pm.StudentT('log_income', nu=nu, mu=μ, sigma=σ, observed=y)

g1 = pm.model_to_graphviz(income_lin_model)
g1

In [ ]:
with income_lin_model:
    trace= pm.sample(random_seed = random_seed, return_inferencedata=False, step = pm.NUTS(target_accept = 0.9))
    income_lin_trace = pm.to_inference_data(trace=trace, log_likelihood = True)

In [ ]:
az.plot_trace(income_lin_trace, kind = 'rank_bars')

In [ ]:
az.summary(income_lin_trace)

In [ ]:
az.plot_posterior(income_lin_trace, ref_val = 0, color="Blue")

In [ ]:
az.plot_forest(income_lin_trace, var_names = ['β'] )

In [ ]:
income_pp = pm.sample_posterior_predictive(income_lin_trace,
                                          model = income_lin_model, random_seed = random_seed)

In [ ]:
az.plot_bpv(income_pp)

In [ ]:
az.plot_ppc(income_pp)

In [ ]:
# Fix to test data
with income_lin_model:
  pm.set_data({'X':X_income_test, 'y': median_income_test_log})
  income_lin_test_pp = pm.sample_posterior_predictive(
      income_lin_trace,
      var_names = ['log_income'],
      return_inferencedata=True,
      predictions=True,
      extend_inferencedata=True,
      random_seed=random_seed
  )

In [ ]:
income_lin_test_pp

In [ ]:
_, ax = plt.subplots(figsize=(8, 4))

pred_mean = income_lin_test_pp.predictions["log_income"].mean(dim=["draw", "chain"])
ax.scatter(X_income_test[0:300,8],pred_mean[0:300], label = 'Predictions')
ax.scatter(X_income_test[0:300,8], median_income_test_log[0:300], label = 'Test Set')
ax.vlines(
    X_income_test[0:300,8],
    *az.hdi(income_lin_test_pp.predictions)["log_income"][0:300].transpose("hdi", ...),
    alpha=0.3,
)
ax.set_ylabel('Log of Median Income')
ax.set_xlabel('Z-scaled Log of Cost of Living')
ax.set_title('Full Model Predictions and Test Set Values')
ax.legend(loc='upper right')

In [ ]:
_, ax = plt.subplots(figsize=(8, 4))

pred_mean = income_lin_test_pp.predictions["log_income"].mean(dim=["draw", "chain"])
ax.scatter(X_income_test[0:350,2],pred_mean[0:350], label = 'Predictions')
ax.scatter(X_income_test[0:350,2], median_income_test_log[0:350], label = 'Test Set')
ax.vlines(
    X_income_test[0:350,2],
    *az.hdi(income_lin_test_pp.predictions)["log_income"][0:350].transpose("hdi", ...),
    alpha=0.3,
)
ax.set_ylabel('Log of Median Income')
ax.set_xlabel('Z-scaled Unemployment Rate')
ax.set_title('Full Model Predictions and Test Set Values')
ax.legend(loc='upper right')

In [ ]:
from sklearn.metrics import mean_squared_error
pred = income_lin_test_pp.predictions["log_income"].mean(dim =['chain','draw'])
mse = mean_squared_error(pred, median_income_test_log)
print("Mean Square Error: %0.2f" % (mse))

##Reduced Model


**Dropped Population, Air Quality, Water Quality, Diversity Rank (Gender), and State Dummy Variables**


In [ ]:
df_reduced = df[['countyhelper', 'ULOCALE', '2016 Crime Rate', 'Unemployment',
       '2020PopulrVoteParty', '2022 Median Income', '2020 PopulrMajor%',
        'Cost of Living', 'Stu:Tea Rank', 'Diversity Rank (Race)']]

df_reduced = pd.get_dummies(df_reduced, drop_first = False,
                              columns = ['ULOCALE'], dtype = float)

df_reduced['2020PopulrVoteParty'] = pd.Categorical(df_reduced['2020PopulrVoteParty']).codes

df_reduced = df_reduced.set_index('countyhelper')

df_reduced.dtypes

In [ ]:
from sklearn.model_selection import train_test_split
train_r, test_r = train_test_split(df_reduced, random_state=2023, test_size=0.2)
train_r

In [ ]:
median_income_train_log_r = np.log(train_r['2022 Median Income'])
median_income_test_log_r = np.log(test_r['2022 Median Income'])
train_r['Cost of Living Log'] = np.log(train_r['Cost of Living'])
test_r['Cost of Living Log'] = np.log(test_r['Cost of Living'])

In [ ]:
train_r.drop(['2022 Median Income', 'Cost of Living'], axis=1, inplace = True)
test_r.drop(['2022 Median Income', 'Cost of Living'], axis=1, inplace = True)

In [ ]:
numeric_var = ['2016 Crime Rate', 'Unemployment', '2020 PopulrMajor%',
               'Cost of Living Log','Stu:Tea Rank', 'Diversity Rank (Race)']
scaler = StandardScaler()
X_scaled_train_r = scaler.fit_transform(train_r[numeric_var])
stats.describe(X_scaled_train_r)

In [ ]:
categorical_var = list(set(train_r.columns) - set(numeric_var))
X_income_train_r = np.hstack((X_scaled_train_r, train_r[categorical_var].to_numpy()))

In [ ]:
X_income_train_r.shape

## Test

In [ ]:
numeric_var = ['2016 Crime Rate', 'Unemployment',
       '2020 PopulrMajor%','Cost of Living Log', 'Stu:Tea Rank', 'Diversity Rank (Race)']
scaler = StandardScaler()
X_scaled_test_r = scaler.fit_transform(test_r[numeric_var])
stats.describe(X_scaled_test_r)

In [ ]:
categorical_var = list(set(test_r.columns) - set(numeric_var ))
X_income_test_r = np.hstack((X_scaled_test_r, test_r[categorical_var].to_numpy()))

In [ ]:
X_income_test_r.shape

In [ ]:
median_income_reduced_mean = median_income_train_log_r.mean()
k = X_income_train_r.shape[1]

with pm.Model() as income_lin_model_reduced:

    #Data
    X = pm.MutableData("X", X_income_train_r) # training data
    y = pm.MutableData("y", median_income_train_log_r)

    #Regression model parameters
    α = pm.Normal('α', mu = median_income_reduced_mean, sigma = 17000) # uninformed prior
    β = pm.MvNormal('β', mu=np.zeros(k), cov= np.eye(k), shape=k, dims ='Predictors') # informed priors

    #Likelihood parameters
    nu = pm.Exponential('nu', 1/29.0)+ 1 # uninformed prior
    σ = pm.HalfStudentT('σ',2,8)
    μ = α + pm.math.dot(X,β)
    log_reduced = pm.StudentT('log_income', nu=nu, mu=μ, sigma=σ, observed=y)

g1 = pm.model_to_graphviz(income_lin_model_reduced)
g1

In [ ]:
with income_lin_model_reduced:
    trace_reduced = pm.sample(random_seed = random_seed, return_inferencedata=False)
    income_lin_trace_reduced = pm.to_inference_data(trace=trace_reduced, log_likelihood = True)

In [ ]:
az.plot_trace(income_lin_trace_reduced, kind = 'rank_bars')

In [ ]:
az.summary(income_lin_trace_reduced)

In [ ]:
az.plot_posterior(income_lin_trace_reduced, ref_val = 0, color="Orange")

In [ ]:
az.plot_forest(income_lin_trace_reduced, var_names = ['β'] )

In [ ]:
income_pp_reduced = pm.sample_posterior_predictive(income_lin_trace_reduced,
                                          model = income_lin_model_reduced, random_seed = random_seed)

In [ ]:
az.plot_bpv(income_pp_reduced)

In [ ]:
az.plot_ppc(income_pp_reduced)

In [ ]:
with income_lin_model_reduced:
  pm.set_data({'X':X_income_test_r, 'y': median_income_test_log_r})
  reduced_model_test_pp = pm.sample_posterior_predictive(
      income_lin_trace_reduced,
      var_names = ['log_income'],
      return_inferencedata=True,
      predictions=True,
      extend_inferencedata=True,
      random_seed=random_seed
  )

In [ ]:
reduced_model_test_pp

In [ ]:
from sklearn.metrics import mean_squared_error
pred = reduced_model_test_pp.predictions["log_income"].mean(dim =['chain','draw'])
mse = mean_squared_error(pred, median_income_test_log_r)
print("Mean Square Error: %0.2f" % (mse))

In [ ]:
_, ax = plt.subplots(figsize=(8, 4))

pred_mean = reduced_model_test_pp.predictions["log_income"].mean(dim=["draw", "chain"])
ax.scatter(X_income_test_r[0:300,3],pred_mean[0:300], label = 'Predictions')
ax.scatter(X_income_test_r[0:300,3], median_income_test_log_r[0:300], label = 'Test Set')
ax.vlines(
    X_income_test_r[0:300,3],
    *az.hdi(reduced_model_test_pp.predictions)["log_income"][0:300].transpose("hdi", ...),
    alpha=0.3,
)
ax.set_ylabel('Log of Median Income')
ax.set_xlabel('Z-scaled Log of Cost of Living')
ax.set_title('Reduced Model Predictions and Test Set Values')
ax.legend(loc='upper right')

In [ ]:
_, ax = plt.subplots(figsize=(8, 4))

pred_mean = reduced_model_test_pp.predictions["log_income"].mean(dim=["draw", "chain"])
ax.scatter(X_income_test_r[0:350,1],pred_mean[0:350], label = 'Predictions')
ax.scatter(X_income_test_r[0:350,1], median_income_test_log_r[0:350], label = 'Test Set')
ax.vlines(
    X_income_test_r[0:350,1],
    *az.hdi(reduced_model_test_pp.predictions)["log_income"][0:350].transpose("hdi", ...),
    alpha=0.3,
)
ax.set_ylabel('Log of Median Income')
ax.set_xlabel('Z-scaled Unemployment Rate')
ax.set_title('Reduced Model Predictions and Test Set Values')
ax.legend(loc='upper right')

## Reduced Non-Linear Model

**Dropped Population, Airquality, Diversity Rank (Gender), and State Dummy Variables**

**Transformed Water Quality (Inverse) and Unemployment (Inverse)**


In [ ]:
nl_df = df[['countyhelper', 'ULOCALE', '2016 Crime Rate', 'Unemployment',
       '2020PopulrVoteParty', 'WaterQualityVPV', '2022 Median Income', '2020 PopulrMajor%',
        'Cost of Living', 'Stu:Tea Rank',
       'Diversity Rank (Race)']]

nl_df = pd.get_dummies(nl_df, drop_first = False,
                              columns = ['ULOCALE'], dtype = float)

nl_df['2020PopulrVoteParty'] = pd.Categorical(nl_df['2020PopulrVoteParty']).codes

nl_df = nl_df.set_index('countyhelper')

nl_df.dtypes

In [ ]:
nl_df['Cost of Living Log'] = np.log(nl_df['Cost of Living'])
nl_df['WaterQualityVPV_inv'] = ((1/(nl_df['WaterQualityVPV']+.01)))
nl_df['Unemployment_inv'] = ((1/(nl_df['Unemployment']+.01)))
nl_df.drop(['Cost of Living', 'Unemployment', 'WaterQualityVPV'], axis=1, inplace = True)

In [ ]:
train_nl, test_nl = train_test_split(nl_df, test_size=0.2, random_state=2023)
median_income_train_log_nl = np.log(train_nl['2022 Median Income'])
median_income_test_log_nl = np.log(test_nl['2022 Median Income'])
train_nl.drop(['2022 Median Income'], axis=1, inplace = True)
test_nl.drop(['2022 Median Income'], axis=1, inplace = True)

In [ ]:
numeric_var = ['2016 Crime Rate', 'Unemployment_inv', '2020 PopulrMajor%',
       'WaterQualityVPV_inv', 'Cost of Living Log',
       'Stu:Tea Rank', 'Diversity Rank (Race)']
scaler = StandardScaler()
X_scaled_train_nl = scaler.fit_transform(train_nl[numeric_var])
stats.describe(X_scaled_train_nl)

In [ ]:
categorical_var = list(set(train_nl.columns) - set(numeric_var ))
X_income_train_nl = np.hstack((X_scaled_train_nl, train_nl[categorical_var].to_numpy()))

In [ ]:
X_income_train_nl.shape

In [ ]:
numeric_var = ['2016 Crime Rate', 'Unemployment_inv', '2020 PopulrMajor%',
       'WaterQualityVPV_inv', 'Cost of Living Log',
       'Stu:Tea Rank', 'Diversity Rank (Race)']
scaler = StandardScaler()
X_scaled_test_nl = scaler.fit_transform(test_nl[numeric_var])
stats.describe(X_scaled_test_nl)

In [ ]:
categorical_var = list(set(test_nl.columns) - set(numeric_var ))
X_income_test_nl = np.hstack((X_scaled_test_nl, test_nl[categorical_var].to_numpy()))

In [ ]:
X_income_test_nl.shape

In [ ]:
median_income_reduced_mean = median_income_train_log_nl.mean()
k = X_income_train_nl.shape[1]

with pm.Model() as income_nl_model_reduced:

    #Data
    X = pm.MutableData("X", X_income_train_nl) # training data
    y = pm.MutableData("y", median_income_train_log_nl)

    #Regression model parameters
    α = pm.Normal('α', mu = median_income_reduced_mean, sigma = 17000) # uninformed prior
    β = pm.MvNormal('β', mu=np.zeros(k), cov= np.eye(k), shape=k, dims ='Predictors') # informed priors

    #Likelihood parameters
    nu = pm.Exponential('nu', 1/29.0)+ 1 # uninformed prior
    σ = pm.HalfStudentT('σ',2,8)
    μ = α + pm.math.dot(X,β)
    log_reduced_nl = pm.StudentT('log_income', nu=nu, mu=μ, sigma=σ, observed=y)

g1 = pm.model_to_graphviz(income_nl_model_reduced)
g1

In [ ]:
with income_nl_model_reduced:
    trace_nl_reduced = pm.sample(random_seed = random_seed, return_inferencedata=False)
    income_nl_trace_reduced = pm.to_inference_data(trace=trace_nl_reduced, log_likelihood = True)

In [ ]:
az.plot_trace(income_nl_trace_reduced , kind = 'rank_bars')

In [ ]:
az.summary(income_nl_trace_reduced)

In [ ]:
az.plot_forest(income_nl_trace_reduced, var_names = ['β'] )

In [ ]:
az.plot_posterior(income_nl_trace_reduced, ref_val = 0, color="Purple")

In [ ]:
income_nl_pp_reduced = pm.sample_posterior_predictive(income_nl_trace_reduced,
                                          model = income_nl_model_reduced, random_seed = random_seed)

In [ ]:
az.plot_bpv(income_nl_pp_reduced)

In [ ]:
az.plot_ppc(income_nl_pp_reduced)

In [ ]:
income_nl_pp_reduced

In [ ]:
with income_nl_model_reduced:
  pm.set_data({'X':X_income_test_nl, 'y': median_income_test_log_nl})
  reduced_model_test_pp_nl = pm.sample_posterior_predictive(
      income_nl_trace_reduced,
      var_names = ['log_income'],
      return_inferencedata=True,
      predictions=True,
      extend_inferencedata=True,
      random_seed=random_seed
  )

In [ ]:
X_income_test_nl

In [ ]:
reduced_model_test_pp_nl

In [ ]:
_, ax = plt.subplots(figsize=(8, 4))

pred_mean = reduced_model_test_pp_nl.predictions["log_income"].mean(dim=["draw", "chain"])
ax.scatter(X_income_test_nl[0:300,4],pred_mean[0:300], label = 'Predictions')
ax.scatter(X_income_test_nl[0:300,4], median_income_test_log_nl[0:300], label = 'Test Set')
ax.vlines(
    X_income_test_nl[0:300,4],
    *az.hdi(reduced_model_test_pp_nl.predictions)["log_income"][0:300].transpose("hdi", ...),
    alpha=0.3,
)
ax.set_ylabel('Log of Median Income')
ax.set_xlabel('Z-scaled Log of Cost of Living')
ax.set_title('Non-Linear Model Predictions and Test Set Values')
ax.legend(loc='upper right')

In [ ]:
_, ax = plt.subplots(figsize=(8, 4))

pred_mean = reduced_model_test_pp_nl.predictions["log_income"].mean(dim=["draw", "chain"])
ax.scatter(X_income_test_nl[0:350,1],pred_mean[0:350], label = 'Predictions')
ax.scatter(X_income_test_nl[0:350,1], median_income_test_log_nl[0:350], label = 'Test Set')
ax.vlines(
    X_income_test_nl[0:350,1],
    *az.hdi(reduced_model_test_pp_nl.predictions)["log_income"][0:350].transpose("hdi", ...),
    alpha=0.3,
)
ax.set_ylabel('Log of Median Income')
ax.set_xlabel('Z-scaled Inverse of Unemployment Rate')
ax.set_title('Non-Linear Model Predictions and Test Set Values')
ax.legend(loc='upper right')

In [ ]:
from sklearn.metrics import mean_squared_error
pred = reduced_model_test_pp_nl.predictions["log_income"].mean(dim =['chain','draw'])
mse = mean_squared_error(pred, median_income_test_log_nl)
print("Mean Square Error: %0.2f" % (mse))

## Bayesian Model Averaging

##LOO Weights

In [ ]:
comp = az.compare({"Standard": income_lin_trace, "Reduced": income_lin_trace_reduced, 'Non-Linear': income_nl_trace_reduced}, ic="loo")
comp

##WAIC Weights

In [ ]:
comp_waic = az.compare({"Standard": income_lin_trace, "Reduced": income_lin_trace_reduced, 'Non-Linear': income_nl_trace_reduced}, ic="waic")
comp_waic

##Averaged Model Predictions Calculated Using LOO Weights

In [ ]:
f = income_lin_test_pp.predictions["log_income"].mean(dim=["draw", "chain"])*(comp.weight.sort_index(ascending=True)[2])

In [ ]:
r = reduced_model_test_pp.predictions["log_income"].mean(dim=["draw", "chain"])*(comp.weight.sort_index(ascending=True)[1])

In [ ]:
nl = reduced_model_test_pp_nl.predictions["log_income"].mean(dim=["draw", "chain"])*(comp.weight.sort_index(ascending=True)[0])

In [ ]:
model_averaged_preds = f+r+nl

In [ ]:
_, ax = plt.subplots(figsize=(8, 4))

ax.scatter(X_income_test_nl[0:300,4],model_averaged_preds[0:300], label = 'Predictions')
ax.scatter(X_income_test_nl[0:300,4], median_income_test_log_nl[0:300], label = 'Test Set')

ax.set_ylabel('Log of Median Income')
ax.set_xlabel('Z-scaled Log of Cost of Living')
ax.set_title('Averaged Model Predictions and Test Set Values')
ax.legend(loc='upper right')

**Limitations: Averaged Model lacks uncertainty due to resources utilized in this study. Google Colab does not utilize the latest version of Arviz, which is necessary to be able to successfully make averaged model predictions using trace information. In addition, the model weights calculated here were determined using pseudo bayesian model averaging, which utilized information criteria to calculate its weights.**

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(model_averaged_preds, median_income_test_log_nl)
print("Mean Square Error: %0.2f" % (mse))

**This model performed better than the full model, though the MSE is still significantly high. This is due to the large weight given to the full model in the overall averaging model.**